In [15]:
import numpy as np
import mmap
from posix_ipc import Semaphore, SharedMemory, ExistentialError
from ctypes import sizeof, memmove, addressof, create_string_buffer
from time import sleep
from ctypes import Structure, c_int32, c_int64, c_char_p, c_char


## Defining the structure that holds the meta-data

In [16]:
class MD(Structure):
    _fields_ = [
        ('byte_size', c_int64),
        ('data_shape', c_int64),
        ('S2_byte_size', c_int32),
        ('S2_shape', c_int32),
    ]
    
md_buf = create_string_buffer(sizeof(MD))

In [17]:
POSIX_name = 'MTL'

sem = Semaphore(POSIX_name, 0)
sem2 = Semaphore(POSIX_name+'2', 0)

In [18]:
shm_buf = None

try:
    md_region = SharedMemory(POSIX_name + '-meta')
    md_buf_ = mmap.mmap(md_region.fd, sizeof(MD))
    md_region.close_fd()
except ExistentialError:
    print("Meta Data shared memory is NOT available.")
    
md = MD()
md_buf[:] = md_buf_
memmove(addressof(md), md_buf, sizeof(md))

for name, xx in md._fields_:
    print(name, xx)

byte_size  = getattr(md, 'byte_size')
data_shape = getattr(md, 'data_shape')

S2_byte_size = getattr(md, 'S2_byte_size')
S2_shape = getattr(md, 'S2_shape')

byte_size <class 'ctypes.c_long'>
data_shape <class 'ctypes.c_long'>
S2_byte_size <class 'ctypes.c_int'>
S2_shape <class 'ctypes.c_int'>


## Please make sure that the MTL file has been already laoded into the POSIX shared memory

In [19]:
# %%timeit -t -n 10 -r 10

shm_buf = None

try:
    md_region = SharedMemory(POSIX_name + '-meta')
    md_buf_ = mmap.mmap(md_region.fd, sizeof(MD))
    shm_region = SharedMemory(name=POSIX_name)
    shm_region2 = SharedMemory(name=POSIX_name+'2')
    md_region.close_fd()
    shm_buf = mmap.mmap(shm_region.fd, byte_size)
    shm_buf2 = mmap.mmap(shm_region2.fd, S2_byte_size)
    shm_region.close_fd()
except ExistentialError:
    print("Data shared memory is NOT available.")

## Decoding the data-type string.
Converting it to a list of tuples

In [20]:
sem2.acquire()

S2 = np.ndarray(shape=(S2_shape,), dtype='int32', buffer=shm_buf2)
sem2.release()

S2[:10]

array([91, 40, 39, 78, 85, 77, 79, 66, 83, 95], dtype=int32)

In [21]:
S = ''.join([chr(x) for x in S2]).split("[")[1].split("]")[0].split(",")

i = 0
dtyle_lst = []
while i < len(S):
    s1 = S[i].split('(')[1].replace("'", "").strip()
    s2 = S[i+1].split(')')[0].replace("'", "").strip()
    
    dtyle_lst.append((s1,s2))
    
    i+=2

dtyle_lst

[('NUMOBS_MORE', '>i8'),
 ('RELEASE', '>i2'),
 ('BRICKID', '>i4'),
 ('BRICKNAME', '<U8'),
 ('BRICK_OBJID', '>i4'),
 ('MORPHTYPE', '<U4'),
 ('RA', '>f8'),
 ('RA_IVAR', '>f4'),
 ('DEC', '>f8'),
 ('DEC_IVAR', '>f4'),
 ('EBV', '>f4'),
 ('FLUX_G', '>f4'),
 ('FLUX_R', '>f4'),
 ('FLUX_Z', '>f4'),
 ('FLUX_IVAR_G', '>f4'),
 ('FLUX_IVAR_R', '>f4'),
 ('FLUX_IVAR_Z', '>f4'),
 ('MW_TRANSMISSION_G', '>f4'),
 ('MW_TRANSMISSION_R', '>f4'),
 ('MW_TRANSMISSION_Z', '>f4'),
 ('FRACFLUX_G', '>f4'),
 ('FRACFLUX_R', '>f4'),
 ('FRACFLUX_Z', '>f4'),
 ('FRACMASKED_G', '>f4'),
 ('FRACMASKED_R', '>f4'),
 ('FRACMASKED_Z', '>f4'),
 ('FRACIN_G', '>f4'),
 ('FRACIN_R', '>f4'),
 ('FRACIN_Z', '>f4'),
 ('NOBS_G', '>i2'),
 ('NOBS_R', '>i2'),
 ('NOBS_Z', '>i2'),
 ('PSFDEPTH_G', '>f4'),
 ('PSFDEPTH_R', '>f4'),
 ('PSFDEPTH_Z', '>f4'),
 ('GALDEPTH_G', '>f4'),
 ('GALDEPTH_R', '>f4'),
 ('GALDEPTH_Z', '>f4'),
 ('FLUX_W1', '>f4'),
 ('FLUX_W2', '>f4'),
 ('FLUX_W3', '>f4'),
 ('FLUX_W4', '>f4'),
 ('FLUX_IVAR_W1', '>f4'),
 ('FLUX_IVA

In [22]:
# %%timeit -t -n 2 -r 2

sem.acquire()

f = np.ndarray(shape=(data_shape,), dtype=dtyle_lst, buffer=shm_buf)
sem.release()


In [23]:
f[:3]


array([(100, 8001, 638477, '2713p682', 1784, 'PSF', 271.57822094, 1.1226929e+14, 68.23066885, 1.1204010e+14, 0.04494829, 13.377395, 12.291068,   8.760663, 245.06502 , 118.429504, 71.51117 , 0.875416 , 0.91427064, 0.9511019, 2.6097623e-04, 1.7119080e-03, 8.2329474e-04, 0.00389323, 4.0909229e-03, 0.01162072, 0.9997574 , 0.9998439 , 0.99894214, 3, 3, 3, 663.42334, 205.63574, 79.37791 , 483.33347, 151.64256, 43.039734,  2.222856,  1.387598, -11.353064 ,   3.7152333, 26.60402 , 4.6205597, 0.02932631, 0.00034518, 0.99241155, 0.99533284, 0.99900275, 0.99962336, 0, 0, 0, 10.414153,  9.5684595,  6.8200793, 10.414153,  9.5684595,  6.8200793, 2015.5, 0, 0, 0, 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2258343419699174784, 'G2', 19.754745, 210.92673, 19.676823,  20.26173, 19.599525,  16.327147, 1.2792153, 0.        , False, 0.5978343 ,  True, 2.4843044 ,   7.7688184, -4.6271515,  2.2121403, 0.49411172,  2.7031229, 'N', 35191448101979896,        8225, 0.59664697, 65535, 3050, 1, 16046,

In [ ]:
# dtype=[('cen', '>i8'), 
#                       ('BA', '>f4'), 
#                       ('TARGETID', '>i8'), 
#                       ('RA', '>f8'), 
#                       ('DEC', '>f8'), 
#                       ('BRICKID', '>i4'), 
#                       ('SGA_ID', '>i8'), 
#                       ('NAME', '<U32'), 
#                       ('NUMOBS_MORE', '>i8'), 
#                       ('OBSCONDITIONS', '>i8'), ('D25', '>f4'), ('PA', '>f4'), ('SUBPRIORITY', '>f8'), ('PRIORITY', '>i8'), ('DESI_TARGET', '>i8')]